# FINM-32000 HW 2
## Carsen Berger
## 2/4/22

In [1]:
import numpy as np

# Problem 1

In [2]:
class Dynamics:
    pass

In [3]:
hw2dynamics = Dynamics()
hw2dynamics.S0 = 100
hw2dynamics.r = 0.06
hw2dynamics.q = 0.01
hw2dynamics.maxvol = 0.6
hw2dynamics.localvol = lambda S, t: np.minimum(0.2 + 5 * np.log(S / 100) ** 2 + 0.1 * np.exp(-t), hw2dynamics.maxvol)

# Note that hw2dynamics.localvol is a function 
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [4]:
class Contract:
    pass

In [5]:
hw2contract = Contract()
hw2contract.putexpiry = 0.75;
hw2contract.putstrike = 95;
hw2contract.callexpiry = 0.25;
hw2contract.callstrike = 10;

In [6]:
class Tree:
    pass

In [7]:
hw2tree = Tree()
hw2tree.N = 3000 #change if necessary to get $0.01 accuracy, in your judgment

In [8]:
def fp_equals(x, y):
    return abs(x - y) <= 1e-12

In [9]:
def pricer_compound_localvol_trinom(contract, dynamics, tree):
    
    put_expiry, put_strike = contract.putexpiry, contract.putstrike
    call_expiry, call_strike = contract.callexpiry, contract.callstrike
    S0, r, q = dynamics.S0, dynamics.r, dynamics.q
    max_vol, local_vol = dynamics.maxvol, dynamics.localvol
    N = tree.N
    
    delta_t = put_expiry / N
    if not fp_equals(call_expiry / delta_t, round(call_expiry / delta_t)):
        raise ValueError("This value of N fails to place the call expiry in the tree.")

    representative_vol = local_vol(S0, 0)
    delta_x = max(representative_vol * np.sqrt(3 * delta_t), max_vol * np.sqrt(delta_t))

    S = S0 * np.exp(np.linspace(N, -N, num=2*N+1, endpoint=True) * delta_x)

    put_price = np.maximum(put_strike - S, 0)

    for t in np.linspace(N-1, 0, num=N, endpoint=True) * delta_t:
        S = S[1:-1]
        sigma = local_vol(S, t)
        
        nu = (r - q) - 0.5 * sigma ** 2
        P_u = 0.5 * ((sigma ** 2 * delta_t + nu ** 2 * delta_t ** 2) / delta_x ** 2 + nu * delta_t / delta_x)
        P_d = 0.5 * ((sigma ** 2 * delta_t + nu ** 2 * delta_t ** 2) / delta_x ** 2 - nu * delta_t / delta_x)
        P_m = 1 - (sigma ** 2 * delta_t + nu ** 2 * delta_t ** 2) / delta_x ** 2

        up_put_prices = put_price[:-2]
        mid_put_prices = put_price[1:-1]
        down_put_prices = put_price[2:]
        
        put_intrinsic_value = np.maximum(put_strike - S, 0)
        put_price = np.maximum(put_intrinsic_value,
                               np.exp(-r * delta_t) * (P_u * up_put_prices + P_m * mid_put_prices + P_d * down_put_prices))
        
        if fp_equals(t, call_expiry):
            call_price = np.maximum(put_price - call_strike, 0)
        elif t < call_expiry:
            up_call_prices = call_price[:-2]
            mid_call_prices = call_price[1:-1]
            down_call_prices = call_price[2:]

            call_price = np.exp(-r * delta_t) * (P_u * up_call_prices + P_m * mid_call_prices + P_d * down_call_prices)
    
    price_of_put = put_price[0]
    
    price_of_call_on_put = call_price[0]
    
    return (price_of_put, price_of_call_on_put)

In [10]:
(answer_part_a, answer_part_b) = pricer_compound_localvol_trinom(hw2contract, hw2dynamics, hw2tree)

In [11]:
answer_part_a, answer_part_b

(7.007352296510587, 1.5925531282199274)

# Problem 2

## 2(a)

The exact formula for the Black-Scholes delta of a vanilla call is

\begin{equation}
\Delta = N(d_1),
\end{equation}

where

\begin{equation}
d_1 = \frac{\log(S / K) + (r + \frac{\sigma^2}{2}) (T - t)}{\sigma \sqrt{T - t}}.
\end{equation}

Plugging in $S = K$ and $t = 0$ yields

\begin{align}
d_1 &= \frac{(r + \frac{\sigma^2}{2}) T}{\sigma \sqrt{T}}\\
&= \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{align}

Now, for $x \approx 0$, to first order,

\begin{equation}
N(x) \approx N(0) + N'(0) \cdot x.
\end{equation}

We know that $N(0) = 0.5$ and $N'(x) = \frac{1}{\sqrt{2 \pi}} \exp(\frac{-x^2}{2})$, so $N'(0) = \frac{1}{\sqrt{2\pi}} \approx 0.4$. Thus,

\begin{align}
\Delta &\approx 0.5 + \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma \sqrt{2 \pi}}\\
&\approx 0.5 + 0.4 \cdot \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{align}

Plugging in $\sigma = 0.2$, $T = 0.25$, and $r = 0.01$ yields

\begin{align}
\Delta &\approx 0.5 + 0.4 \cdot \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}\\
&\approx 0.5 + 0.4 \cdot \frac{(0.01 + \frac{0.2^2}{2}) \sqrt{0.25}}{0.2}\\
&\approx 0.5 + 0.4 \cdot \frac{(0.01 + 0.02) 0.5}{0.2}\\
&\approx 0.5 + 0.4 \cdot \frac{0.03 \cdot 0.5}{0.2}\\
&\approx 0.5 + 0.4 \cdot \frac{0.015}{0.2}\\
&\approx 0.5 + 0.4 \cdot 0.075\\
&\approx 0.5 + 0.03\\
&\approx 0.53.
\end{align}

## 2(b)

Let $\Delta_\$$ and $\Gamma_\$$ denote the dollar delta and dollar gamma, respectively.

We are given $S_0 = 4$, $\Delta_\$ = 3$, $\Gamma_\$ = 0.02$, and $C(S_0) = 5$.

We find that

\begin{align}
\Delta_\$ = 3 &\implies S_0 \Delta = 3\\
&\implies 4 \Delta = 3\\
&\implies \Delta = \frac{3}{4}
\end{align}

and

\begin{align}
\Gamma_\$ = 0.02 &\implies \frac{1}{100} S_0^2 \Gamma = 0.02\\
&\implies S_0^2 \Gamma = 2\\
&\implies 4^2 \Gamma = 2\\
&\implies 16 \Gamma = 2\\
&\implies \Gamma = \frac{1}{8}.
\end{align}

Now, for $S \approx S_0$, to second order, we have

\begin{align}
C(S) &\approx C(S_0) + \Delta (S - S_0) + \frac{\Gamma}{2} (S - S_0)^2\\
&\approx 5 + \frac{3}{4} (S - 4) + \frac{1}{16} (S - 4)^2.
\end{align}

For $S = 3.6$, we have

\begin{align}
C(3.6) &\approx 5 + \frac{3}{4} (3.6 - 4) + \frac{1}{16} (3.6 - 4)^2\\
&\approx 5 + \frac{3}{4} (-0.4) + \frac{1}{16} (-0.4)^2\\
&\approx 5 - 0.3 + \frac{1}{16} \cdot 0.16\\
&\approx 5 - 0.3 + 0.01\\
&\approx 4.71.
\end{align}